# Crater Detection using YOLOv8

In this notebook, we will use the YOLOv8 object detection model to detect martian and lunar craters. The dataset will be downloaded from kaggle, and we will use the ultralytics implementation of YOLOv8.

## Setup

In [1]:
! pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip install -q kaggle

### Kaggle setup to download the dataset

Kaggle allows you to create an API token, and use it to download datasets using the command line. We use that token here, download and unzip the dataset, and we can use it as normal.

In [3]:
from google.colab import files
files.upload()
print("done")

Saving kaggle.json to kaggle.json
done


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d lincolnzh/martianlunar-crater-detection-dataset

 84% 62.0M/74.0M [00:00<00:00, 209MB/s]
100% 74.0M/74.0M [00:00<00:00, 214MB/s]


In [6]:
!unzip martianlunar-crater-detection-dataset.zip
print("done")

Archive:  martianlunar-crater-detection-dataset.zip
  inflating: best.pt                 
  inflating: craters/test/images/010_png.rf.fcf5e274562ee69a325f9d7a0b30767f.jpg  
  inflating: craters/test/images/015_png.rf.7d5b2091b6339c9480a171a59c52c3b9.jpg  
  inflating: craters/test/images/019_png.rf.1930cd277f9bf0e3fa57f2dcfee0385f.jpg  
  inflating: craters/test/images/04_png.rf.81a7d6cbeb9dc09e5a8ecd40e185fc92.jpg  
  inflating: craters/test/images/mars_crater--100-_jpg.rf.a2ad5867efb2d73e86d9d980ca40a9fe.jpg  
  inflating: craters/test/images/mars_crater--108-_jpg.rf.9395f473f249e064dbfea078e2519a17.jpg  
  inflating: craters/test/images/mars_crater--116-_jpg.rf.2e550a693a8800808e68848484716b95.jpg  
  inflating: craters/test/images/mars_crater--117-_jpg.rf.b412b6593d102c5f9dda7bce79bb815c.jpg  
  inflating: craters/test/images/mars_crater--12-_jpg.rf.26060e7ca4ca8781bcaf011b9b70b7db.jpg  
  inflating: craters/test/images/mars_crater--25-_jpg.rf.a06bfa24b404b064ead471f56d636e0e.jpg  

## Dataset setup

In [7]:
!rm -rf /content/craters/working/
!mkdir -p /content/craters/working/images /content/craters/working/labels

!cp -a /content/craters/train/images/. /content/craters/working/images
!cp -a /content/craters/train/labels/. /content/craters/working/labels
!cp -a /content/craters/valid/images/. /content/craters/working/images
!cp -a /content/craters/valid/labels/. /content/craters/working/labels

To direct the model to the dataset, we must create a yaml config file containing the dataset information. The config file should look like this:

```yaml
path: /path/to/dataset
train: train/images # path to train images
val: val/images # path to validation images
test: test/images # path to test images

# Classes
names:
    0: crater
```

Here, the yaml file is in the root directory, and is named `data_config.yaml`

In [8]:
import os
import yaml

data_config = {"path": "/content/craters", "train":"working/images", "val":"test/images", "names":{0:"crater"}}

with open("/content/data_config.yaml","w") as yamlfile:
  yaml.dump(data_config, yamlfile, default_flow_style=False)


## Training the model

In [9]:
from ultralytics import YOLO

In [10]:
# model = YOLO("yolov8n.yaml") # trains model from scratch
model = YOLO("yolov8s.yaml")

In [11]:
results = model.train(data="/content/data_config.yaml", batch=8, epochs=25, single_cls=True, optimizer='SGD')

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/content/data_config.yaml, epochs=25, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

100%|██████████| 755k/755k [00:00<00:00, 14.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 77.1MB/s]


AMP: checks passed ✅


...

      25/25      2.33G      2.541        2.8      3.044         14        640: 100%|██████████| 16/16 [00:02<00:00,  5.43it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.82it/s]

                   all         19        151      0.306      0.225      0.193     0.0721



25 epochs completed in 0.032 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.07it/s]


                   all         19        151      0.309      0.225      0.193     0.0721
Speed: 0.3ms preprocess, 5.0ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to runs/detect/train


## Zip the results and download to local system

In [12]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [13]:
!zip -r /content/runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/results.csv (deflated 84%)
  adding: content/runs/detect/train/events.out.tfevents.1714482747.f2c4d3e41901.249.0 (deflated 91%)
  adding: content/runs/detect/train/val_batch1_pred.jpg (deflated 7%)
  adding: content/runs/detect/train/labels_correlogram.jpg (deflated 44%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 38%)
  adding: content/runs/detect/train/weights/ (stored 0%)
  adding: content/runs/detect/train/weights/best.pt (deflated 10%)
  adding: content/runs/detect/train/weights/last.pt (deflated 10%)
  adding: content/runs/detect/train/args.yaml (deflated 52%)
  adding: content/runs/detect/train/train_batch240.jpg (deflated 7%)
  adding: content/runs/detect/train/val_batch1_labels.jpg (deflated 7%)
  adding: content/runs/detect/train/train_batch242.jpg (deflated 5%)
  adding: content/runs/detect/

In [14]:
files.download("/content/runs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
files.download("/content/yolov8n.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Prediction Testing using file upload

- Save a copy of the notebook so you have the results and training parameters

- Turn off GPU and switch to CPU runtime

- We will upload the model and the image for prediction

In [ ]:
# upload image
from google.colab import files
files.upload()
print("done")

Saving mars_crater--63-_jpg.rf.ab0440022651f886c68744a55eb215d9.jpg to mars_crater--63-_jpg.rf.ab0440022651f886c68744a55eb215d9.jpg
done


In [ ]:
# upload model
from google.colab import files
files.upload()
print("done")

Saving best.pt to best.pt
done


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.1/749.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# Prediction

In [ ]:
from ultralytics import YOLO
import cv2

# if continuing with the same runtime

# choose which training run's model you would like to use
# train_run = "train" # every time the model gets trained, a new directory with
# the weights and results is created as train1, train2, etc.

# Load a model
# model = YOLO(f'/content/runs/detect/{train_run}/weights/best.pt')  # pretrained YOLOv8n model

# else
model = YOLO('/content/best.pt')

# make prediction
result = model.predict('/content/mars_crater--63-_jpg.rf.ab0440022651f886c68744a55eb215d9.jpg', conf = 0.2, save=True, show=True)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


image 1/1 /content/mars_crater--63-_jpg.rf.ab0440022651f886c68744a55eb215d9.jpg: 640x640 (no detections), 1115.2ms
Speed: 3.8ms preprocess, 1115.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict8


In [ ]:
!zip -r /content/runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/predict4/ (stored 0%)
  adding: content/runs/detect/predict4/mars_crater--63-_jpg.rf.ab0440022651f886c68744a55eb215d9.jpg (deflated 7%)
  adding: content/runs/detect/predict5/ (stored 0%)
  adding: content/runs/detect/predict5/mars_crater--63-_jpg.rf.ab0440022651f886c68744a55eb215d9.jpg (deflated 7%)
  adding: content/runs/detect/predict/ (stored 0%)
  adding: content/runs/detect/predict/05_png.rf.844343145246e51e66a345419e1862bf.jpg (deflated 5%)
  adding: content/runs/detect/predict3/ (stored 0%)
  adding: content/runs/detect/predict3/mars_crater--29-_jpg.rf.719ff6d6c1aafd71147c173f7d224018.jpg (deflated 5%)
  adding: content/runs/detect/predict2/ (stored 0%)
  adding: content/runs/detect/predict2/mars_crater--31-_jpg.rf.5030dfd316db1570d5f2ad396fa87d4d.jpg (deflated 6%)


In [ ]:
files.download('/content/runs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>